In [12]:
import requests
import pandas as pd

In [13]:
# api-endpoint
SET = 'bro'
URL = "https://api.scryfall.com/cards/search?q=set="+SET


In [14]:
def mount_cards_df(URL, lista):
    card_attr_list = lista
    r = requests.get(url = URL)
    data = r.json()
    aux_df = pd.DataFrame()
    #Read json
    for card in data['data']:

        if len(card['color_identity']) == 0:
            color_identity = 'T'
        elif len(card['color_identity']) > 1:
            color_identity = 'M'
        else:
            color_identity = card['color_identity']

        if len(card['colors']) == 0:
            colors = 'T'
        elif len(card['colors']) > 1:
            colors = 'M'
        else:
            colors = card['colors']
        card_attr_list.append(aux_df)
        
        aux_dict = { 'name': [card['name']],
                    'rarity': [card['rarity']],
                    'color_identity': color_identity,
                    'colors': colors,
                }
        card_attr_list.append(pd.DataFrame.from_dict(aux_dict))

    if data['has_more']:
        next_page = data['next_page']
        mount_cards_df(next_page, card_attr_list)

    return card_attr_list

In [10]:
card_attr_list = mount_cards_df(URL,[])
card_attr_df = pd.concat(card_attr_list)

In [15]:
card_attr_df.to_csv('Data/all_cards.csv', index=False)